# PyCI Tutorial

PyCI is a flexible quantum chemistry Configuration Interaction library for Python 3.

It supports pair-occupied spatial orbital (DOCI), orthogonal spin-orbital (FullCI), and generalized orbital (GenCI) wavefunction symmetries through its main classes:

`pyci.doci_wfn`

`pyci.fullci_wfn`

`pyci.genci_wfn`

Restricted and generalized Hamiltonians are explicitly supported with the main Hamiltonian class:

`pyci.hamiltonian`

Unrestricted Hamiltonians can be constructed as a special case of generalized Hamiltonian.

Combined with routines to control the type of configurations added, according to their excitation level or seniority, PyCI offers the the ability to program any CI method.

Here we will look at the following wavefunction models using **Be** as our toy model:
* Full CI
* CISD
* GKCI
* HCI
* Seniority-truncated CI

Lets start showing how to run a Full CI calculation

In [1]:
# Import modules

import pyci
# optional
import numpy as np
from pyci.test import datafile

In [2]:
# System information
filename = datafile("be_ccpvdz.fcidump")
occs = (2,2)
ham = pyci.hamiltonian(filename)
e_dict = {}

In [3]:
# Make an empty wave function instance and add all possible determinants to it
wfn = pyci.fullci_wfn(ham.nbasis, *occs)
wfn.add_all_dets()
# Solve the CI matrix problem
op = pyci.sparse_op(ham, wfn)
e_vals, e_vecs = op.solve(n=1, tol=1.0e-9)
e_dict["Full-CI"] = (len(wfn), e_vals[0])

Next we will see how to truncate the Full CI space based on excitation level or seniority number of the Slater determinants.

In [4]:
# Create a CISD wave function
excitations = (0, 1, 2)
wfn2 = pyci.fullci_wfn(ham.nbasis, *occs)
pyci.add_excitations(wfn2, *excitations, ref=None)

op = pyci.sparse_op(ham, wfn2)
e_vals, e_vecs2 = op.solve(n=1, tol=1.0e-9)
e_dict["CISD"] = (len(wfn2), e_vals[0])

In [5]:
# Defining a seniority truncated CI wave function
seniorities = [0, 2]
wfn3 = pyci.fullci_wfn(ham.nbasis, *occs)
pyci.add_seniorities(wfn3, *seniorities)

op = pyci.sparse_op(ham, wfn3)
e_vals, e_vecs3 = op.solve(n=1, tol=1.0e-9)
e_dict["SenTrunc-CI"] = (len(wfn3), e_vals[0])

The Griebel-Knapeck and Heat-bath Configuration Interaction models are also supported in PyCI.

* Griebel-Knapeck CI

In [6]:
wfn4 = pyci.fullci_wfn(ham.nbasis, *occs)
pyci.add_gkci(wfn4, t=-0.5, p=1.0, mode="cntsp", dim=3, energies=None, width=None)

op = pyci.sparse_op(ham, wfn4)
e_vals, e_vecs4 = op.solve(n=1, tol=1.0e-9)
e_dict["GKCI"] = (len(wfn4), e_vals[0])

* Heat Bath CI

In [7]:
wfn5 = pyci.fullci_wfn(ham.nbasis, *occs)

# Add Hartree-Fock determinant
wfn5.add_hartreefock_det()
dets_added = 1

# Create CI matrix operator and initial Hartree-Fock solution
op = pyci.sparse_op(ham, wfn5)
e_vals, e_vecs5 = op.solve(n=1, tol=1.0e-9)

# Run HCI iterations
niter = 0
eps = 5.0e-4
while dets_added:
    # Add connected determinants to wave function via HCI
    dets_added = pyci.add_hci(ham, wfn5, e_vecs5[0], eps=eps)
    # Update CI matrix operator
    op.update(ham, wfn5)
    # Solve CI matrix problem
    e_vals, e_vecs5 = op.solve(n=1, tol=1.0e-9)
    niter += 1
e_dict["HCI"] = (len(wfn5), e_vals[0])

In [8]:
print(f"{'Model':<15} {'# Dets':>10} {'Energy [a.u.]':>10}")
for key in e_dict:
    print(f"{key:<15} {e_dict[key][0]:>10} {e_dict[key][1]:>10.5f}")

Model               # Dets Energy [a.u.]
Full-CI               8281  -14.61741
CISD                   757  -14.61736
SenTrunc-CI           2275  -14.61723
GKCI                   169  -14.61684
HCI                    282  -14.61740


Finally, we demostrate how to compute:
- the overlap with the Full CI solution 
- the spin-resolved 1 and 2-RDMs

In [9]:
ovl = pyci.compute_overlap(wfn, wfn, e_vecs, e_vecs)
ovl2 = pyci.compute_overlap(wfn2, wfn, e_vecs2, e_vecs)
ovl3 = pyci.compute_overlap(wfn3, wfn, e_vecs3, e_vecs)
ovl4 = pyci.compute_overlap(wfn4, wfn, e_vecs4, e_vecs)
ovl5 = pyci.compute_overlap(wfn5, wfn, e_vecs5, e_vecs)

print(f"<FCI|FCI>  = {ovl:.5f}")
print(f"<CISD|FCI> = {ovl2:.5f}")
print(f"<SCI|FCI>  = {ovl3:.5f}")
print(f"<GKCI|FCI> = {ovl4:.5f}")
print(f"<HCI|FCI>  = {ovl5:.5f}")

<FCI|FCI>  = 1.00000
<CISD|FCI> = 1.00000
<SCI|FCI>  = -0.99998
<GKCI|FCI> = 0.99995
<HCI|FCI>  = 1.00000


In [10]:
d1, d2 = pyci.compute_rdms(wfn5, e_vecs5[0])
rdm1, rdm2 = pyci.spinize_rdms(d1, d2)

nelec = np.trace(rdm1)
npairs = np.einsum('pqpq', rdm2) / 2.0

print(f"Number of electrons = {nelec:.1f}")
print(f"Number of pairs     = {npairs:.1f}")

Number of electrons = 4.0
Number of pairs     = 6.0
